In [1]:
from astropy.cosmology import FlatLambdaCDM
from time import clock
from os import path
import glob, os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib
from astropy.table import Table, Column

In [2]:
def flux2L(flux,z):
    """Transfer flux to luminoity assuming a flat Universe"""
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    DL = cosmo.luminosity_distance(z).value*10**6*3.08*10**18 # unit cm
    L = flux*1.e-17*4.*np.pi*DL**2   #erg/s/A
    return L

def L_conti(wave,flux,z):
    return np.asarray([np.log10(5100*flux2L(flux[np.where( abs(wave - 5100.) < 5.,True,False)].mean(),z) )])

In [3]:
tie_balmer=False
limit_doublets=False
ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

mass_frac_all = np.array([])
ml_all = np.array([])
L_all = np.array([])
zbin3_syst_vel = []

In [4]:
for i in zip(glob.glob('/Users/ANguyen\MyWorks\zbin3_specs_sv\spec-*.fits')):
    #print (np.shape(i))
    hdu = fits.open(i[0])
    t = hdu[1].data
    wv = 10**t['loglam']
    redshift = float(hdu[2].data['z'])
    #print ('redshift:',np.round(redshift,4))
    
    wave1 = 10**t['loglam']/(1.+ redshift)
    flux1 = t['flux']
    err1 = 1./np.sqrt(t['ivar'])
    z = 0.
    mask = np.where( (wave1 > 3541.) & (wave1 < 7409.) & (err1 <100.), True, False)

    flux = flux1[mask]
    galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
    factor1=np.median(flux)
    #print 'flux normalized by:', np.median(flux)
    wave = wave1[mask]
    err = err1[mask]
    wave *= np.median(util.vac_to_air(wave)/wave)
    noise = np.full_like(galaxy, 0.01635)  # Assume constant noise per pixel here
    c = 299792.458  # speed of light in km/s
    velscale = c*np.log(wave[1]/wave[0])  # eq.(8) of Cappellari (2017)
    FWHM_gal = 2.76  # SDSS has an approximate instrumental resolution FWHM of 2.76A.
    pathname = ppxf_dir + '/miles_models/Mun1.30*.fits'
    miles = lib.miles(pathname, velscale, FWHM_gal)
    reg_dim = miles.templates.shape[1:]
    stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)

    regul_err = 0.01  # Desired regularization error

    lam_range_gal = np.array([np.min(wave), np.max(wave)])/(1. + z)
    gas_templates, gas_names, line_wave = util.emission_lines(
        miles.log_lam_temp, lam_range_gal, FWHM_gal,
        tie_balmer=tie_balmer, limit_doublets=limit_doublets)
    templates = np.column_stack([stars_templates, gas_templates])

    c = 299792.458
    dv = c*(miles.log_lam_temp[0] - np.log(wave[0]))  # eq.(8) of Cappellari (2017)
    vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
    start = [vel, 180.]  # (km/s), starting guess for [V, sigma]

    n_temps = stars_templates.shape[1]
    n_forbidden = np.sum(["[" in a for a in gas_names])  # forbidden lines contain "[*]"
    n_balmer = len(gas_names) - n_forbidden
    component = [0]*n_temps + [1]*n_balmer + [2]*n_forbidden
    gas_component = np.array(component) > 0  # gas_component=True for gas templates
    moments = [4, 4, 4]

    start = [start, start, start]

    gas_reddening = 0 if tie_balmer else None
    t = clock()
    
    len_galaxy = len(galaxy)
    len_template = len(templates)
    difference = len_galaxy - len_template

    if len_galaxy > len_template:
        galaxy = galaxy[:len(galaxy)-difference]
        noise = noise[:len(noise)-difference]
        wave = noise[:len(wave)-difference]
        
    elif len_galaxy == len_template:
        galaxy = galaxy[:len(galaxy)-1]
        noise = noise[:len(noise)-1]
        wave = noise[:len(wave)-1]
    else:
        galaxy = galaxy
        noise = noise
        
    pp = ppxf(templates, galaxy, noise, velscale, start,
              plot=False, moments=moments, degree=-1, mdegree=10, vsyst=dv,
              lam=wave, clean=False, regul=1./regul_err, reg_dim=reg_dim,
              component=component, gas_component=gas_component,
              gas_names=gas_names, gas_reddening=gas_reddening)
    
    syst_vel = c*np.log(1 + redshift) + pp.sol[1][0]
    
    print ('syst_vel',syst_vel)
    
    zbin3_syst_vel.append(syst_vel)
    
    #print('Desired Delta Chi^2: %.4g' % np.sqrt(2*galaxy.size))
    #print('Current Delta Chi^2: %.4g' % ((pp.chi2 - 1)*galaxy.size))

      

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -274       455    -0.031     0.034
 comp.  1:        22       235    -0.133     0.300
 comp.  2:       -62       261     0.031    -0.118
chi2/DOF: 1394; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 165; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  30 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.4695     0.067      22   235
Comp:  1        Hgamma     0.6636     0.067      22   235
Comp:  1         Hbeta     0.4763     0.066      22   235
Comp:  1        Halpha      15.41     0.066      22   235
Comp:  2     [OII]3726          0       0.1     -62   261
Comp:  2     [OII]3729      14.51       0.1     -62   261
Comp:  2     [SII]6716      3.745     0.056     -62   261
Comp:  2     [SII]6731          0     0.056     -62   26

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       100       197     0.148    -0.215
 comp.  1:       252       209    -0.042    -0.070
 comp.  2:       306       161     0.012    -0.116
chi2/DOF: 329.2; degree = -1; mdegree = 10
method = capfit; Jac calls: 12; Func calls: 278; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  50 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta          0     0.053     252   209
Comp:  1        Hgamma     0.1014     0.053     252   209
Comp:  1         Hbeta     0.7451     0.053     252   209
Comp:  1        Halpha      5.267     0.052     252   209
Comp:  2     [OII]3726      1.484     0.063     306   161


...

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       -75       156    -0.048     0.166
 comp.  1:      -107       183    -0.016     0.015
 comp.  2:       -83       233    -0.018    -0.014
chi2/DOF: 21.81; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 212; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  51 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta          0     0.053    -107   183
Comp:  1        Hgamma     0.1238     0.053    -107   183
Comp:  1         Hbeta     0.5483     0.053    -107   183
Comp:  1        Halpha      1.263     0.052    -107   183
Comp:  2     [OII]3726     0.2595       0.1     -83   233
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        75       207     0.123    -0.014
 comp.  1:       130       268    -0.076    -0.014
 comp.  2:        42       328    -0.117    -0.056
chi2/DOF: 540.3; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 186; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  36 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.4885     0.062     130   268
Comp:  1        Hgamma      2.652     0.061     130   268
Comp:  1         Hbeta      2.558     0.061     130   268
Comp:  1        Halpha      7.971     0.062     130   268
Comp:  2     [OII]3726      8.945      0.14      42   328
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       154       179    -0.013    -0.008
 comp.  1:       338       209    -0.050    -0.123
 comp.  2:       187       153     0.202    -0.009
chi2/DOF: 173.2; degree = -1; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 257; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  39 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.3756     0.051     338   209
Comp:  1        Hgamma          0     0.051     338   209
Comp:  1         Hbeta      1.394     0.051     338   209
Comp:  1        Halpha      15.73      0.05     338   209
Comp:  2     [OII]3726      1.752     0.066     187   153


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       101       206     0.014    -0.193
 comp.  1:       203       167    -0.063    -0.079
 comp.  2:       164       185    -0.048     0.013
chi2/DOF: 440.3; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 188; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  27 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta          0     0.048     203   167
Comp:  1        Hgamma          0     0.048     203   167
Comp:  1         Hbeta      2.218     0.048     203   167
Comp:  1        Halpha      13.87     0.047     203   167
Comp:  2     [OII]3726      4.782     0.082     164   185
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -254       321    -0.014     0.001
 comp.  1:      -216       708    -0.074    -0.020
 comp.  2:       -24       232    -0.078     0.045
chi2/DOF: 3358; degree = -1; mdegree = 10
method = capfit; Jac calls: 12; Func calls: 280; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  11 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      3.094     0.098    -216   708
Comp:  1        Hgamma       3.61     0.097    -216   708
Comp:  1         Hbeta      11.77     0.097    -216   708
Comp:  1        Halpha      47.47      0.11    -216   708
Comp:  2     [OII]3726      9.468      0.11     -24   232
C

In [5]:
zbin3_syst_vel_array = np.array(zbin3_syst_vel)

In [6]:
col0 = fits.Column(name = 'SYST_VEL', format = 'D', array = zbin3_syst_vel_array)

coldefs = fits.ColDefs([col0])
hdu = fits.BinTableHDU.from_columns(coldefs)

zbin3_sv = Table(hdu.data)

In [7]:
(zbin3_sv).write('zbin3_sv.fits')